# Data Exploration - Team Gambinos

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
complaints = pd.read_csv('../data/complaints.csv')

We decided as a team to adopt the following naming convention for consistency:

In [25]:
complaints.columns = [x.replace('Consumer ', '').lower().replace(' ', '_') for x in complaints.columns]
complaints['complaint_length'] = complaints['complaint_narrative'].apply(len)


In [26]:
complaints['issue'].value_counts()

Incorrect information on your report    229305
Attempts to collect debt not owed        73163
Communication tactics                    21243
Struggling to pay mortgage               17374
Fraud or scam                            12347
Name: issue, dtype: int64

In [27]:
complaints_dict = {'Attempts to collect debt not owed': 1, 
                 'Communication tactics': 2, 
                 'Fraud or scam': 3, 
                 'Incorrect information on your report': 4,
                 'Struggling to pay mortgage': 5}

In [28]:
# to replace the text in issue with target variables
complaints = complaints.replace({'issue': complaints_dict})

In [29]:
complaints['complaint_narrative']

0         My name is XXXX XXXX this complaint is not mad...
1         I searched on XXXX for XXXXXXXX XXXX  and was ...
2         I have a particular account that is stating th...
3         I have not supplied proof under the doctrine o...
4         Hello i'm writing regarding account on my cred...
                                ...                        
353427           Collections account I have no knowledge of
353428    Dear CFPB Team, The reason for my complaint is...
353429    FRCA violations : Failing to Follow Debt Dispu...
353430    My Father, a XXXX XXXX  acquired an HECM rever...
353431    I have tried to contact cash app about a fraud...
Name: complaint_narrative, Length: 353432, dtype: object

In [30]:
seed = 123
for complaint in complaints.loc[complaints['issue'] == 4, 'complaint_narrative'].sample(3, random_state=seed):
    print(complaint)
    print('-----------------------------')

I just reviewed a copy of my Experian credit report and found the following information to be an error. I am a victim of identity theft and did not make these charges. I reported the theft of my identity to the Federal Trade Commission.
-----------------------------
after my legal separation from my husband he started to open credit in my name with no permission I have a legal case against him but can not find him he has ruined my life his name is XXXX XXXX XXXX   remove I have disputed by mail with all three credit bureaus. act # XXXX owing an alleged {$2200.00} XXXX   XXXX  orig creditor XXXX
-----------------------------
Equifax report a collections account " XXXX XXXX XXXX ''. I have no knowledge of this account. I have not been furnished any proof or verifications of this account. I don't have any signed contract agreements with XXXX XXXX XXXX, verbal agreements nor any paperwork associated with them. A proper investigation according to FCRA was not conducted, word of mouth from t

In [31]:
# remove double x-s
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace(r'[xX][xX]+','', regex=True)

# stripping all numeric values from complaints
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace('\d+', '', regex=True)

# removing digit removes "4927" words

In [32]:
#complaints.groupby('issue')['complaint_narrative'].apply(lambda x: x.str.extractall(r'(\w+X+\w+)').groupby(0).size().reset_index(name='count'))

In [33]:
complaints.loc[complaints['complaint_narrative'].str.find('TOXIC') != -1, 'complaint_narrative'].iloc[3]

'  and      , CA  NATIONSTAR/.   #  We need immediate assistance with our   property loan.     is the investor of the property and   is the servicer for the loan. \n\nOur property has a domestic water well and the water is unsafe to drink. The significant and very serious water contaminations in the aquifers that serve our water well include : Lead Hexavalent Chromium ( documented in the movie entitled   ) and Arsenic. These chemicals are listed on the CA Proposition  List of Contaminants that Cause Cancer or Reproductive Harm. Serial testing of the water well by a CA Certified Laboratory defined the contamination to be steady-state as we were informed by the US EPA. There is no filtration system that can reduce these high levels down to potable safe drinking water levels. Our home is now defined* as : HAZARDOUS TOXIC CONTAMINATED UNINHABITABLE POLLUTED NON-POTABLE WATER BLIGHTED AND A NUISANCE ( *as defined by the CA List of Contaminants Proposition  State of CA, EPA Office of Environ

In [34]:
X = complaints[['complaint_narrative']]
y = complaints['issue']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

In [35]:
# note: the default stop words remove 311 words; have to specify it, otherwise it's off
vect = CountVectorizer(lowercase=True, stop_words='english', ) 

#r”(?u)\b\w\w+\b”

X_train_vec = vect.fit_transform(X_train['complaint_narrative'])
X_test_vec = vect.transform(X_test['complaint_narrative'])

In [36]:
vect.vocabulary_

{'filed': 22553,
 'identity': 27952,
 'theft': 56640,
 'report': 47749,
 'learned': 32873,
 'using': 60926,
 'incarceration': 28928,
 'submitted': 54607,
 'documentation': 17959,
 'transunion': 58380,
 'disregarded': 17555,
 'didn': 16614,
 'block': 7279,
 'account': 684,
 'days': 14777,
 'receiving': 45983,
 'information': 29632,
 'received': 45955,
 'letter': 33156,
 'irs': 31244,
 'stating': 53836,
 'tax': 55882,
 'return': 48889,
 'dept': 15992,
 'educatio': 19000,
 'acct': 816,
 'closed': 10666,
 'send': 51009,
 'written': 63727,
 'notification': 38526,
 'reported': 47762,
 'consumer': 12569,
 'reporting': 47789,
 'agencies': 2006,
 'late': 32640,
 'updated': 60655,
 'chargeoff': 9817,
 'providing': 44621,
 'severe': 51387,
 'scandalous': 50336,
 'action': 1122,
 'destroying': 16295,
 'perfect': 41791,
 'payment': 41410,
 'history': 26947,
 'credit': 13871,
 'battle': 6245,
 'going': 25215,
 'submittig': 54612,
 'cfpb': 9627,
 'complaint': 11711,
 'delete': 15520,
 'following': 23

In [37]:
len(vect.vocabulary_)

64231

### Naive Bayes First

In [39]:
word_counts = pd.DataFrame({
    'words': vect.get_feature_names_out(),
    'frequency': np.array(X_train_vec.sum(axis = 0)).flatten()
})

word_counts.head()

,words,frequency
0,__,24
1,___,48
2,____,44
3,_____,31
4,______,4


In [40]:
word_counts.sort_values('frequency', ascending=False).head(25)

,words,frequency
13871,credit,552596
684,account,375715
47749,report,314043
29632,information,290369
47789,reporting,213057
14948,debt,174366
12569,consumer,152087
734,accounts,134206
5825,balance,115379
11590,company,112763


In [41]:
from sklearn.naive_bayes import MultinomialNB

In [42]:
nb = MultinomialNB().fit(X_train_vec, y_train)

y_pred = nb.predict(X_test_vec)

In [43]:
accuracy_score(y_test, y_pred)

0.826467326105163

In [44]:
confusion_matrix(y_test, y_pred)

array([[11809,  1985,   446,  3765,   286],
       [  597,  4474,    54,   105,    81],
       [   65,    54,  2817,   119,    32],
       [ 4770,   705,   668, 49686,  1497],
       [   26,    30,    16,    32,  4239]])

### LogReg

In [45]:
logreg = LogisticRegression(max_iter = 10000).fit(X_train_vec, y_train)

y_pred = logreg.predict(X_test_vec)

accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)

array([[12095,   790,   119,  5223,    64],
       [  956,  3966,    23,   336,    30],
       [  145,    33,  2653,   237,    19],
       [ 2587,   124,    95, 54363,   157],
       [  107,    38,    28,   334,  3836]])

In [ ]:
vect = TfidfVectorizer()

X_train_vec = vect.fit_transform(X_train['text'])
X_test_vec = vect.transform(X_test['text'])

In [ ]:
logreg = LogisticRegression(max_iter = 10000).fit(X_train_vec, y_train)

y_pred = logreg.predict(X_test_vec)

accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)